In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = PhysNet(TN=True)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss=np_loss)
model(np.random.random((4, 128, 32, 32, 3)));
model.summary()

Model: "phys_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (4, 128, 32, 32, 16)   │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (4, 128, 16, 16, 32)   │        13,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 128, 16, 16, 64)   │        55,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_9 (Sequential)       │ (4, 64, 2, 2, 64)      │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_10 (Sequential)      │ (4, 128, 2, 2, 64)     │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_9 (Conv3D)               │ (4, 128, 1, 1, 1)      │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (4, 128, 16, 16, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (4, 64, 8, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling3d               │ (4, 128, 1, 1, 64)     │             0 │
│ (AveragePooling3D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (4, 128)               │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,825 (2.94 MB)

 Trainable params: 768,577 (2.93 MB)

 Non-trainable params: 1,248 (4.88 KB)

In [3]:
batch_size = 128

tape = "/root/ssd_cache/rppg_training_data/mmpd_128x32x32_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)

mmpd_128x32x32_all             datatape has been loaded.    10368 items total. fold=train selected
mmpd_128x32x32_all             datatape has been loaded.     2176 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/physnettn_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/physnettn_mmpd.weights.h5')
model.load_weights('../weights/physnettn_mmpd.weights.h5')

Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.9407	Validation loss:0.8193	


Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1274	Validation loss:0.7849	


Training:   0%|          | 0/81 [00:00<?, ?it/s]

Validating:   0%|          | 0/17 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1154	Validation loss:0.7857	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_MMPD_RLAP.h5', scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/PhysNetTN_MMPD_RLAP.h5')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,2.446±0.087,6.677±1.789,3.41±0.141,0.81754
Whole,1.32±0.159,2.7±1.36,1.817±0.235,0.96856


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_MMPD_COHFACE.h5', fps=30)
get_metrics('../results/PhysNetTN_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,3.025±0.153,7.205±2.291,4.446±0.239,0.81423
Whole,2.042±0.228,3.563±1.71,3.032±0.35,0.95539


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_MMPD_PURE.h5')
get_metrics('../results/PhysNetTN_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,4.275±0.454,13.235±4.898,6.79±0.761,0.82397
Whole,3.93±0.984,8.522±5.127,6.21±1.618,0.93964


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNetTN_MMPD_UBFC.h5')
get_metrics('../results/PhysNetTN_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.648±0.163,4.067±2.391,1.663±0.154,0.97128
Whole,0.879±0.205,1.592±1.089,0.86±0.194,0.99616
